# Data preparation

In [1]:
import os
import numpy as np
np.random.seed(666)
import torch
import torchvision
import torchvision.datasets as datasets
if not os.path.isdir('data'):
    ! mkdir data

In [4]:
cifar10_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
cifar10_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
x_train = np.array([np.array(img[0])/255. for img in cifar10_trainset])
y_train = np.array([img[1] for img in cifar10_trainset])
x_test = np.array([np.array(img[0])/255. for img in cifar10_testset])
y_test = np.array([img[1] for img in cifar10_testset])

np.savez('data/cifar10', arr_0=x_train, arr_1=y_train, arr_2=x_test, arr_3=y_test)

In [2]:
# load images as queries

dataset = np.load('data/cifar10.npz')
x_train = dataset['arr_0']
y_train = dataset['arr_1']
x_test = dataset['arr_2']
y_test = dataset['arr_3']

print("training data shape: {}, {}, testing data shape: {}, {}".format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))

training data shape: (50000, 32, 32, 3), (50000,), testing data shape: (10000, 32, 32, 3), (10000,)


# Start Blacklight detection

In [3]:
from probabilistic_fingerprint import *

In [4]:
# set parameters for blacklight
window_size = 20
hash_kept = 50
roundto = 50
step_size = 1
workers = 5
threshold = 25

In [5]:
def get_tracker(query, window_size, hash_kept, roundto, step_size, workers):
    tracker = InputTracker(query, window_size, hash_kept, round=roundto, step_size=step_size, workers=workers)
    LOGGER.info("Blacklight detector created.")
    return tracker

In [6]:
tracker = get_tracker(x_train[0], window_size, hash_kept, roundto, step_size, workers)
match_list = []
id = 0
for query in x_test:
    match_num = tracker.add_img(query)
    match_list.append(match_num)
    print("Image: {}, max match: {}, attack_query: {}".format(id, match_num, match_num>threshold))
    id += 1

[16:55:59,06/07 <ipython-input-5-4804658fbdbf>:3 INFO]: Blacklight detector created.


Image: 0, max match: 0, attack_query: False
Image: 1, max match: 0, attack_query: False
Image: 2, max match: 0, attack_query: False
Image: 3, max match: 0, attack_query: False
Image: 4, max match: 0, attack_query: False
Image: 5, max match: 0, attack_query: False
Image: 6, max match: 0, attack_query: False
Image: 7, max match: 0, attack_query: False
Image: 8, max match: 0, attack_query: False
Image: 9, max match: 0, attack_query: False
Image: 10, max match: 0, attack_query: False
Image: 11, max match: 0, attack_query: False
Image: 12, max match: 1, attack_query: False
Image: 13, max match: 0, attack_query: False
Image: 14, max match: 0, attack_query: False
Image: 15, max match: 0, attack_query: False
Image: 16, max match: 0, attack_query: False
Image: 17, max match: 0, attack_query: False
Image: 18, max match: 1, attack_query: False
Image: 19, max match: 0, attack_query: False
Image: 20, max match: 0, attack_query: False
Image: 21, max match: 0, attack_query: False
Image: 22, max match

In [7]:
np.sum([1 for i in match_list if i > threshold])

0.0